1. Cargue la información del archivo large_even.json en una lista, muestre la cantidad de registros total (deben ser 746, 909). Este es nuestro tráfico inicial

In [2]:
import json

ruta_archivo = 'large_eve.json'
datos = []

with open(ruta_archivo, 'r') as archivo:
    for linea in archivo:
        try:
            datos.append(json.loads(linea))
        except json.JSONDecodeError as e:
            print("Error al decodificar una línea:", e)


print("Cantidad de registros:", len(datos))



Cantidad de registros: 746909


2. Debido a que estamos buscando dominios web, del total de registros, solamente estamos interesados en los registros DNS. Cargue únicamente aquellos registros que sean DNS

In [ ]:



dns_registros = [registro for registro in datos if registro.get("event_type") == "dns"]
print(f"Registros DNS: {len(dns_registros)}")


for i in range(2):
    print(json.dumps(dns_registros[i], indent=4))


Total de registros cargados: 746909
Registros DNS: 15749
{
    "timestamp": "2017-07-22T17:33:16.661646-0500",
    "flow_id": 1327836194150542,
    "pcap_cnt": 22269,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
    "src_port": 59680,
    "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 15529,
        "rrname": "api.wunderground.com",
        "rrtype": "A",
        "tx_id": 0
    }
}
{
    "timestamp": "2017-07-22T17:33:24.990320-0500",
    "flow_id": 2022925111925872,
    "pcap_cnt": 54352,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
    "src_port": 38051,
    "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 58278,
        "rrname": "stork79.dropbox.com",
        "rrtype": "A",
       

3. Muestre la nueva cantidad de registros filtrados. Deben ser 21484. Esta es una cantidad
mucho más manejable, pero aún se debe seguir depurando la información a buscar

In [5]:
dns_registros = []

for registro in datos:
    try:
        if registro.get("event_type") == "dns" and "dns" in registro:
            dns_registros.append(registro)
    except Exception:
        continue

print(f"Cantidad de registros DNS: {len(dns_registros)}")  # Debería ser 21484


Cantidad de registros DNS: 15749


4. Muestre la información de 2 registros cualesquiera

In [ ]:
import json

for i in range(2):
    print(json.dumps(dns_registros[i], indent=4))


{
    "timestamp": "2017-07-22T17:33:16.661646-0500",
    "flow_id": 1327836194150542,
    "pcap_cnt": 22269,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
    "src_port": 59680,
    "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 15529,
        "rrname": "api.wunderground.com",
        "rrtype": "A",
        "tx_id": 0
    }
}
{
    "timestamp": "2017-07-22T17:33:24.990320-0500",
    "flow_id": 2022925111925872,
    "pcap_cnt": 54352,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
    "src_port": 38051,
    "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
    "dest_port": 53,
    "proto": "UDP",
    "dns": {
        "type": "query",
        "id": 58278,
        "rrname": "stork79.dropbox.com",
        "rrtype": "A",
        "tx_id": 0
    }
}


5.  Debido a que la data consiste en estructuras JSON anidadas, utilice la característica
json_normalize para normalizar la información y asignarla en un dataframe. Muestre el shape
del dataframe, debería obtener (21484, 163)

In [ ]:
import pandas as pd
from pandas import json_normalize


df_dns = json_normalize(dns_registros)

print("Shape del dataframe:", df_dns.shape)

df_dns.head()


Shape del dataframe: (15749, 18)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


6. Como estamos buscando dominios DGA, debemos filtrar los registros DNS para aquellos
registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio). Después
de filtrar debería obtener 2849 registros.
